## Example course catalog crawler 

In [32]:
import urllib.request
import json
import csv
import re
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import re
import pandas as pd
import numpy as np
import nltk 
import linkGrabber
from nltk import word_tokenize
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import urllib.parse

In [78]:
#keyword preprocessing
def preprocess(keyword):
    keyword = keyword.lower() #lowercase
    keyword = word_tokenize(keyword) #tokenize
    for word in keyword:
        keyword = stemmer.stem(word) #stem 
        #keyword = lemmatizer.lemmatize(word) #lemmatize
    return (keyword)

#course catalog URLs - 2 academic years 
urls = ['https://classes.cornell.edu/search/roster/FA17?q=',
        'https://classes.cornell.edu/search/roster/SP18?q=',
        'https://classes.cornell.edu/search/roster/SU18?q=',
        'https://classes.cornell.edu/search/roster/FA18?q=',
        'https://classes.cornell.edu/search/roster/SP19?q=',
        'https://classes.cornell.edu/search/roster/SU19?q=']
        
#import keywords
keywords = pd.read_csv("keywords.csv")
technical = keywords[(keywords['Technical/Normative']=='T') & (keywords['Include']=='Y')].Keyword
normative = keywords[(keywords['Technical/Normative']=='N') & (keywords['Include']=='Y')].Keyword
normative = [preprocess(i) for i in normative]
technical = [preprocess(i) for i in technical] 

#replace keywords of interest
normative = [w.replace('privaci', 'privac') for w in normative]
normative = [w.replace('democraci', 'democra') for w in normative]
normative = [w.replace('equiti', 'equit') for w in normative]
normative = [w.replace('histori', 'histor') for w in normative]
normative = [w.replace('justice', 'justic') for w in normative]
normative = [w.replace('liberti', 'libert') for w in normative]
normative = [w.replace('philosophi', 'philosoph') for w in normative]
normative = [w.replace('societi', 'societ') for w in normative]
normative = [w.replace('polici', 'polic') for w in normative]

technical = [w.replace('ai', '^ai') for w in technical]
technical = [w.replace('cs', '^cs') for w in technical]
technical = [w.replace('ict', '^ict') for w in technical]
technical = [w.replace('ml', '^ml') for w in technical]
technical = [w.replace('nlp', '^nlp') for w in technical]

print(normative)
print(technical)

['account', 'critic', 'democra', 'discrimin', 'equal', 'equit', 'ethic', 'fair', 'femin', 'gender', 'govern', 'histor', 'inequ', 'justic', 'law', 'legal', 'libert', 'moral', 'norm', 'philosoph', 'polit', 'power', 'privac', 'race', 'religi', 'respons', 'right', 'secur', 'social', 'societ', 'surveil', 'transpar', 'valu', 'polic']
['^ai', 'algorithm', 'analyt', 'intellig', 'automat', 'code', 'comput', '^cs', 'cyber', 'data', 'digit', '^ict', 'inform', 'intelligen', 'internet', 'machin', '^ml', 'process', '^nlp', 'platform', 'program', 'robot', 'softwar', 'system', 'technolog']


In [71]:
#init df
special = pd.DataFrame(columns=['title','dept_num','description','credits','instructor',
                                'syllabus','university','term','keyword'])

#roster search for all urls
for url in urls:
    #loop through all normative words and extract relevant elements 
    for word in normative: 
        url_keyword = url + word
        response = requests.get(url_keyword)
        soup = BeautifulSoup(response.content, 'lxml')
        #extract relevant elements 
        titles = [p.get_text() for p in soup.select(".title-coursedescr")] #get full desc
        dept_nums = [p.get_text() for p in soup.select('.title-subjectcode')]
        descs = [p.get_text() for p in soup.select('.course-descr')]
        credit = [p.get_text() for p in soup.select('.credit-val')]
        profs = [p.get_text() for p in soup.select('.instructors span.tooltip-iws')]
        #TODO: extract syllabi link
        syllabi = [p.get_text() for p in soup.select('.enrlgrp-syllabi')] 
        syllabi = [p.replace('Syllabi:\n','') for p in syllabi] 
        uni = ['cornell university']
        term = [btn.get_text().strip() for btn in soup.select('button',{'class':'btn btn-default dropdown-toggle'})[:1]]       
        keyword = [word]
        #pull all instances of words we want and append to data in 'special' df
        if word=='privac' or word=='secur':
            for a,b,c,d,e,f,g,h,i in zip(titles,dept_nums,descs,credit,profs,syllabi,uni,term,keyword):
                special = special.append({'title': a, 
                                        'dept_num': b,
                                        'description': c,
                                        'credits': d,
                                        'instructor': e,
                                        'syllabus': f,
                                        'university': g,
                                        'term': h,
                                        'keyword': i}, ignore_index=True)
special

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword
0,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2017,privac
1,Fixed-Income Securities,AEM 4260,Focuses on fixed-income securities including c...,4,"Bogan, V",none,cornell university,Fall 2017,secur
2,Privacy in the Digital Age,CS 5436,This course introduces students to privacy tec...,3-4,"Nissenbaum, H",none,cornell university,Spring 2018,privac
3,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2018,secur
4,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2018,privac
5,National Security Affairs / Preparation for Ac...,AIRS 4401,This course is designed for college seniors an...,3,"Heath, M",none,cornell university,Fall 2018,secur
6,"Internet Law, Privacy and Security",LAW 6568,"This is a survey course in Internet law, with ...",3,"Grimmelmann, J",1 available,cornell university,Spring 2019,privac
7,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2019,secur


In [80]:
#init df
cornell = pd.DataFrame(columns=['title','dept_num','description','credits','instructor',
                                'syllabus','university','term','keyword'])

#roster search for all urls
for url in urls:
    #loop through all normative words and extract relevant elements 
    for word in normative: 
        url_keyword = url + word
        response = requests.get(url_keyword)
        soup = BeautifulSoup(response.content, 'lxml')
        #extract relevant elements 
        titles = [p.get_text() for p in soup.select(".title-coursedescr")] #get full desc
        dept_nums = [p.get_text() for p in soup.select('.title-subjectcode')]
        descs = [p.get_text() for p in soup.select('.course-descr')]
        credit = [p.get_text() for p in soup.select('.credit-val')]
        profs = [p.get_text() for p in soup.select('.instructors span.tooltip-iws')]
        #TODO: extract syllabi link
        syllabi = [p.get_text() for p in soup.select('.enrlgrp-syllabi')] 
        syllabi = [p.replace('Syllabi:\n','') for p in syllabi] 
        uni = ['cornell university']
        term = [btn.get_text().strip() for btn in soup.select('button',{'class':'btn btn-default dropdown-toggle'})[:1]]       
        keyword = [word]
        #filter courses containing >=1 technical and normative keyword            
        for a,b,c,d,e,f,g,h,i in zip(titles,dept_nums,descs,credit,profs,syllabi,uni,term,keyword):
            for word in technical:
                if word in a:
                    print(word)
                    cornell = cornell.append({'title': a, 
                        'dept_num': b,
                        'description': c,
                        'credits': d,
                        'instructor': e,
                        'syllabus': f,
                        'university': g,
                        'term': h,
                        #import union of terms
                        'keyword': [i,keyword]}, ignore_index=True)

cornell

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword


In [81]:
#combine dfs 
cornell = pd.concat([cornell, special])
cornell

#De-duplicate cross listed courses
# cornell.drop_duplicates('title', inplace=True)

,title,dept_num,description,credits,instructor,syllabus,university,term,keyword
0,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2017,privac
1,Fixed-Income Securities,AEM 4260,Focuses on fixed-income securities including c...,4,"Bogan, V",none,cornell university,Fall 2017,secur
2,Privacy in the Digital Age,CS 5436,This course introduces students to privacy tec...,3-4,"Nissenbaum, H",none,cornell university,Spring 2018,privac
3,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2018,secur
4,Security and Privacy Concepts in the Wild,CS 5435,This course will impart a technical and social...,3,"Juels, A",1 available,cornell university,Fall 2018,privac
5,National Security Affairs / Preparation for Ac...,AIRS 4401,This course is designed for college seniors an...,3,"Heath, M",none,cornell university,Fall 2018,secur
6,"Internet Law, Privacy and Security",LAW 6568,"This is a survey course in Internet law, with ...",3,"Grimmelmann, J",1 available,cornell university,Spring 2019,privac
7,Practitioner's Overview of Securities Markets ...,AEM 3060,A broad overview of various aspects of the Fix...,1,"Edwards, A",1 available,cornell university,Spring 2019,secur


In [82]:
#export as csv
cornell.to_csv('cornell.csv')